In [ ]:
import os
os.environ["RAY_USE_MULTIPLE_GPUS"] = "0"
os.environ["RAY_NUM_CPUS"] = "2"
os.environ["TUNE_DISABLE_STRICT_METRIC_CHECKING"] = "1"

In [ ]:
from ultralytics import YOLO
import torch

In [ ]:
if torch.cuda.is_available():
    device = 0
    print(f"Training on GPU")
else:
    print(f"Training on CPU")
    device = "cpu"

In [ ]:
"""
Default values used for all YOLO trainings.

These parameters should remain consistent across different training runs,
unless the hardware or training context changes significantly.
"""

model = YOLO("yolo11n.pt")

default_args = {
    "data": "config/materials.yaml",     # dataset configuration path
    "project": "material_koulutus",      # output base folder
    "imgsz": 640,                     # default input size for YOLO
    "batch": 8,                      # maximum batch size (GPU memory limit)
    "device": 0,                      # GPU device ID
    "cache": "disk",                   # cache data in RAM for speed
    "workers": 1,                     # parallel data loader workers
    "amp": True,                      # automatic mixed precision (faster training)
    "close_mosaic": 10,               # disable mosaic in last N epochs
    "save": True,                     # save checkpoints and weights
    "val": True,                      # enable validation after each epoch
    "plots": True                     # save training/validation plots
}

In [ ]:
from ray import tune

def train_yolo(config):
    from ray.air import session
    model = YOLO("yolo11n.pt")
    results = model.train(
        data="/home/jovyan/Projektit/projekti-3/YOLO/config/materials.yaml",
        epochs=30,
        imgsz=640,
        batch=8,
        device=0,
        lr0=config["lr0"],
        momentum=config["momentum"],
        weight_decay=config["weight_decay"],
        val=True,
        save=True,
        project="/home/jovyan/Projektit/projekti-3/YOLO/runs/detect",
        name=session.get_trial_name(),
        workers=1,
        verbose=False
    )
    map_value = results.results_dict.get("metrics/mAP50-95(B)", 0.0)
    tune.report(metrics={"final_map50_95": map_value})

search_space = {
    "lr0": tune.uniform(1e-4, 1e-2),
    "momentum": tune.uniform(0.7, 0.95),
    "weight_decay": tune.uniform(0.0, 0.001)
}

tuner = tune.Tuner(
    tune.with_resources(train_yolo, {"cpu": 1, "gpu": 0.5}),
    param_space=search_space,
    tune_config=tune.TuneConfig(
        metric="final_map50_95",
        mode="max",
        num_samples=6,
        max_concurrent_trials=2
    ),
    run_config=tune.RunConfig(name="train_minna_raytune_p")
)

tune_results = tuner.fit()

In [ ]:
df = pd.read_csv(results_file)
print(df.columns.tolist())

In [ ]:
import pandas as pd

detect_dir = "/home/jovyan/Projektit/projekti-3/YOLO/runs/detect"
all_data = []

for trial in os.listdir(detect_dir):
    trial_path = os.path.join(detect_dir, trial)
    best_pt_path = os.path.join(trial_path, "weights", "best.pt")
    results_file = os.path.join(trial_path, "results.csv")

    if (
        os.path.isdir(trial_path)
        and trial.startswith("train_yolo_c24f9_")
        and os.path.exists(best_pt_path)
        and os.path.exists(results_file)
    ):
        df = pd.read_csv(results_file)
        if not df.empty and "metrics/mAP50(B)" in df.columns:
            best_row = df.loc[df["metrics/mAP50(B)"].idxmax()]
            all_data.append({
                "trial_name": trial,
                "best_pt_path": best_pt_path,
                "epoch": int(best_row["epoch"]),
                "mAP50": best_row["metrics/mAP50(B)"],
                "precision": best_row.get("metrics/precision(B)", None),
                "recall": best_row.get("metrics/recall(B)", None),
            })

df_summary = pd.DataFrame(all_data)
df_summary.sort_values("mAP50", ascending=False, inplace=True)
df_summary